In [2]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

In [3]:
# read data
df = pd.read_csv('../data/feature_importance_neg.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [4]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
# gridsearch
for i in [128, 256, 512]:  # layer
    for p in ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']:  # optimizers
        tf.set_random_seed(42)

        # make keras model
        start = time.time()
        inputs = Input(shape=(X_train.shape[1],))
        x = Dense(i, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)
        
        # compile
        model.compile(
            loss='categorical_crossentropy',
            optimizer=p,
            metrics=['accuracy']
        )

        epochs = 100
        batch_size = 1000
        es = EarlyStopping(monitor='val_loss', patience=20)

        history = model.fit(
            X_train,
            y_train_for_keras,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test_for_keras),
            verbose=0,
            callbacks=[
                es,
            ]
        )
        
        print(i, p, model.evaluate(X_test, y_test_for_keras, verbose=0)[1])
        
        if model.evaluate(X_test, y_test_for_keras, verbose=0)[1] > load_model('../model/Keras_fs_neg.h5').evaluate(X_test, y_test_for_keras, verbose=0)[1]:
            model.save('../model/Keras_fs_neg.h5')

128 SGD 0.5319148923488374
128 RMSprop 0.5851063893196431
128 Adagrad 0.6914893591657598
128 Adadelta 0.5744680927154866
128 Adam 0.5851063817105395
128 Adamax 0.574468083838199
128 Nadam 0.6489361676763981
256 SGD 0.5638297859658586
256 RMSprop 0.6382978786813452
256 Adagrad 0.6382978698040577
256 Adadelta 0.6382978799495291
256 Adam 0.6170212740593768
256 Adamax 0.6170212740593768
256 Nadam 0.65957447569421
512 SGD 0.5638297859658586
512 RMSprop 0.6702127646892628
512 Adagrad 0.6702127735665504
512 Adadelta 0.6170212842048483
512 Adam 0.6170212740593768
512 Adamax 0.6382978698040577
512 Nadam 0.6382978698040577


In [5]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(
    len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adamax(),
    metrics=['accuracy']
)

epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
    ]
)

model.save('../model/Keras_fs_neg.h5')

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 4ms/step - loss: 2.3687 - acc: 0.2145 - val_loss: 1.2681 - val_acc: 0.4255
Epoch 2/100
373/373 [==============================] - 0s 82us/step - loss: 1.9805 - acc: 0.3110 - val_loss: 1.3050 - val_acc: 0.4681
Epoch 3/100
373/373 [==============================] - 0s 48us/step - loss: 1.9940 - acc: 0.2788 - val_loss: 1.3774 - val_acc: 0.5000
Epoch 4/100
373/373 [==============================] - 0s 64us/step - loss: 1.6994 - acc: 0.3405 - val_loss: 1.3839 - val_acc: 0.5319
Epoch 5/100
373/373 [==============================] - 0s 41us/step - loss: 1.7107 - acc: 0.3861 - val_loss: 1.3570 - val_acc: 0.5319
Epoch 6/100
373/373 [==============================] - 0s 49us/step - loss: 1.6111 - acc: 0.3968 - val_loss: 1.2787 - val_acc: 0.5426
Epoch 7/100
373/373 [==============================] - 0s 47us/step - loss: 1.5617 - acc: 0.3914 - val_loss: 1.2046 - val_acc: 0.5532
Epoch 8/100
373/37

Epoch 62/100
373/373 [==============================] - 0s 45us/step - loss: 0.8075 - acc: 0.6408 - val_loss: 0.9130 - val_acc: 0.6170
Epoch 63/100
373/373 [==============================] - 0s 44us/step - loss: 0.8150 - acc: 0.6327 - val_loss: 0.9109 - val_acc: 0.6277
Epoch 64/100
373/373 [==============================] - 0s 50us/step - loss: 0.8060 - acc: 0.6515 - val_loss: 0.9089 - val_acc: 0.6277
Epoch 65/100
373/373 [==============================] - 0s 48us/step - loss: 0.7637 - acc: 0.6542 - val_loss: 0.9067 - val_acc: 0.6277
Epoch 66/100
373/373 [==============================] - 0s 48us/step - loss: 0.8676 - acc: 0.6488 - val_loss: 0.9045 - val_acc: 0.6277
Epoch 67/100
373/373 [==============================] - 0s 50us/step - loss: 0.7900 - acc: 0.6434 - val_loss: 0.9026 - val_acc: 0.6383
Epoch 68/100
373/373 [==============================] - 0s 44us/step - loss: 0.8009 - acc: 0.6622 - val_loss: 0.8999 - val_acc: 0.6489
Epoch 69/100
373/373 [==============================] -

0.7083333333333334